# Fragestellung 1.3: Welche Gebiete in einer Gemeinde sind am stärksten betroffen?

Um diese Frage zu beantworten, werden Daten aus dem "Gebäude- und Wohnungsregister" (GWR) verwendet. <br>
[Link zum GWR](https://www.housing-stat.ch/de/index.html)<br>
[Link zur GWR Dokumentation](https://www.housing-stat.ch/de/docs/index.html)<br>
[Link zum GWR Download](https://www.housing-stat.ch/de/madd/public.html)<br>
<br>
Die Daten werden nach Gemeinde und Energiequelle gruppiert und dann die einzelnen Objekte gezählt. <br>
Anschliessend wird die resultierende Liste als Geopackage zur weiterverarbeitung bzw. Visualisierung exportiert.<br>
<br>
---
<i> CAS Spatial Data Analytics 2022 </i> ¦ <i> Kommunale Übersicht von Heizsystemen und Energieträgern in Wohngebäuden </i> ¦ <i> Stand: 22.09.2022  </i> ¦ <i> Entwickler: Jürg Reist </i>

### Notebook vorbereiten und die benötigten Daten aus dem GWR einlesen

In [11]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

# Datenquelle definieren
url = urlopen("https://public.madd.bfs.admin.ch/ch.zip")

#lesen GWR Daten
zipfile = ZipFile(BytesIO(url.read()))
dfGWRSource = pd.read_csv(zipfile.open('gebaeude_batiment_edificio.csv'),
                         usecols=['GGDENAME', 'GKODE', 'GKODN', 'GENH1'],
                         sep='\t')

dfGWRSource.head()


GGDENAME        GKODE        GKODN   GENH1
0  Aeugst am Albis  2679647.268  1237500.347  7530.0
1  Aeugst am Albis  2680635.900  1236936.229  7530.0
2  Aeugst am Albis  2679016.828  1235828.156  7530.0
3  Aeugst am Albis  2679677.869  1237453.256  7530.0
4  Aeugst am Albis  2679692.997  1237491.284  7530.0

In [12]:
#Column mit geeigneten Namen
dfGWRSource = dfGWRSource.rename(columns={"GGDENAME": "Gemeinde",
                                          "GKODE": "lat",
                                          "GKODN": "lon",
                                          "GENH1":"Energiequelle"
                                          })
dfGWRSource.head()

Gemeinde          lat          lon  Energiequelle
0  Aeugst am Albis  2679647.268  1237500.347         7530.0
1  Aeugst am Albis  2680635.900  1236936.229         7530.0
2  Aeugst am Albis  2679016.828  1235828.156         7530.0
3  Aeugst am Albis  2679677.869  1237453.256         7530.0
4  Aeugst am Albis  2679692.997  1237491.284         7530.0

In [13]:
# Die Bezeichnungen den Heizcodes zuweisen und als Liste speichern.
# Um die finale Visualisierung auf die drei definierten Energiequellen (Gas, Heizöl, Elektrizität) zu fokussieren, werden die Energiequellen zusammengefasst.
# Im GWR-Datenfile sind folgende Energiequellen definiert:
# 7501: 'Luft', 7510: 'Erdwärme (generisch', 7511: 'Erdwärmesonde', 7512: 'Erdregister', 7513: 'Wasser (Grundwasser, Oberflächenwasser, Abwasser)'
# 7520: 'Gas', 7530: 'Heizöl', 7540: 'Holz (generisch)', 7541: 'Holz (Stückholz)', 7542: 'Holz (Pellets)', 7543: 'Holz (Schnitzel)', 7550: 'Abwärme (innerhalb des Gebäudes)'
# 7560: 'Elektrizität', 7570: 'Sonne (thermisch)', 7580: 'Fernwärme (generisch)', 7581: 'Fernwärme (Hochtemperatur)', 7582: 'Fernwärme (Niedertemperatur)'
# 7598: 'Unbestimmt', 7599: 'Andere'

dfGWRSource['Energiequelle'] = dfGWRSource.Energiequelle.replace({
                                            7500: 'Keine',
                                            7501: 'Weitere',
                                            7510: 'Weitere',
                                            7511: 'Weitere',
                                            7512: 'Weitere',
                                            7513: 'Weitere',
                                            7520: 'Gas',
                                            7530: 'Heizöl',
                                            7540: 'Weitere',
                                            7541: 'Weitere',
                                            7542: 'Weitere',
                                            7543: 'Weitere',
                                            7550: 'Weitere',
                                            7560: 'Elektrizität',
                                            7570: 'Weitere',
                                            7580: 'Weitere',
                                            7581: 'Weitere',
                                            7582: 'Weitere',
                                            7598: 'Unbestimmt',
                                            7599: 'Weitere'                                           
                                            })
dfGWRSource.head()

Gemeinde          lat          lon Energiequelle
0  Aeugst am Albis  2679647.268  1237500.347        Heizöl
1  Aeugst am Albis  2680635.900  1236936.229        Heizöl
2  Aeugst am Albis  2679016.828  1235828.156        Heizöl
3  Aeugst am Albis  2679677.869  1237453.256        Heizöl
4  Aeugst am Albis  2679692.997  1237491.284        Heizöl

In [14]:
dfGWRSource

Gemeinde          lat          lon Energiequelle
0        Aeugst am Albis  2679647.268  1237500.347        Heizöl
1        Aeugst am Albis  2680635.900  1236936.229        Heizöl
2        Aeugst am Albis  2679016.828  1235828.156        Heizöl
3        Aeugst am Albis  2679677.869  1237453.256        Heizöl
4        Aeugst am Albis  2679692.997  1237491.284        Heizöl
...                  ...          ...          ...           ...
3060059       La Baroche  2582768.637  1251359.261        Heizöl
3060060       La Baroche  2582743.072  1251406.135        Heizöl
3060061       La Baroche  2584600.501  1251495.804           Gas
3060062       La Baroche  2583393.948  1251448.443       Weitere
3060063       La Baroche  2582634.389  1251094.914       Weitere

[3060064 rows x 4 columns]

In [15]:
#nur die fossilen Energiequellen filtern
options = ['Gas', 'Heizöl', 'Elektrizität']
dfGWREnergiequelleGHE = dfGWRSource[(dfGWRSource['Energiequelle'].isin(options))] 

dfGWREnergiequelleGHE

                                                 

Gemeinde          lat          lon Energiequelle
0        Aeugst am Albis  2679647.268  1237500.347        Heizöl
1        Aeugst am Albis  2680635.900  1236936.229        Heizöl
2        Aeugst am Albis  2679016.828  1235828.156        Heizöl
3        Aeugst am Albis  2679677.869  1237453.256        Heizöl
4        Aeugst am Albis  2679692.997  1237491.284        Heizöl
...                  ...          ...          ...           ...
3060052       La Baroche  2583088.040  1251351.424        Heizöl
3060058       La Baroche  2583355.208  1251442.906        Heizöl
3060059       La Baroche  2582768.637  1251359.261        Heizöl
3060060       La Baroche  2582743.072  1251406.135        Heizöl
3060061       La Baroche  2584600.501  1251495.804           Gas

[1251114 rows x 4 columns]

### Daten als Geopackage zur Weiterverarbeitung bzw. Visusliserung exportieren

In [16]:
gdf = gpd.GeoDataFrame(
    dfGWREnergiequelleGHE, geometry=gpd.points_from_xy(dfGWREnergiequelleGHE.lat, dfGWREnergiequelleGHE.lon)
)

In [17]:
gdf

Gemeinde          lat          lon Energiequelle  \
0        Aeugst am Albis  2679647.268  1237500.347        Heizöl   
1        Aeugst am Albis  2680635.900  1236936.229        Heizöl   
2        Aeugst am Albis  2679016.828  1235828.156        Heizöl   
3        Aeugst am Albis  2679677.869  1237453.256        Heizöl   
4        Aeugst am Albis  2679692.997  1237491.284        Heizöl   
...                  ...          ...          ...           ...   
3060052       La Baroche  2583088.040  1251351.424        Heizöl   
3060058       La Baroche  2583355.208  1251442.906        Heizöl   
3060059       La Baroche  2582768.637  1251359.261        Heizöl   
3060060       La Baroche  2582743.072  1251406.135        Heizöl   
3060061       La Baroche  2584600.501  1251495.804           Gas   

                                geometry  
0        POINT (2679647.268 1237500.347)  
1        POINT (2680635.900 1236936.229)  
2        POINT (2679016.828 1235828.156)  
3        POINT (2679677.869 1237453.256)  
4        POINT (2679692.997 1237491.284)  
...                                  ...  
3060052  POINT (2583088.040 1251351.424)  
3060058  POINT (2583355.208 1251442.906)  
3060059  POINT (2582768.637 1251359.261)  
3060060  POINT (2582743.072 1251406.135)  
3060061  POINT (2584600.501 1251495.804)  

[1251114 rows x 5 columns]

In [10]:
#Datenframe für Verteilung der Energietrager pro Gemeinde
dfEnergyHeatmap = gdf[['Gemeinde','Energiequelle', 'geometry']]
dfEnergyHeatmap

Gemeinde Energiequelle                         geometry
0        Aeugst am Albis        Heizöl  POINT (2679647.268 1237500.347)
1        Aeugst am Albis        Heizöl  POINT (2680635.900 1236936.229)
2        Aeugst am Albis        Heizöl  POINT (2679016.828 1235828.156)
3        Aeugst am Albis        Heizöl  POINT (2679677.869 1237453.256)
4        Aeugst am Albis        Heizöl  POINT (2679692.997 1237491.284)
...                  ...           ...                              ...
3053211       La Baroche        Heizöl  POINT (2583088.040 1251351.424)
3053217       La Baroche        Heizöl  POINT (2583355.208 1251442.906)
3053218       La Baroche        Heizöl  POINT (2582768.637 1251359.261)
3053219       La Baroche        Heizöl  POINT (2582743.072 1251406.135)
3053220       La Baroche           Gas  POINT (2584600.501 1251495.804)

[1252661 rows x 3 columns]

### Daten in ein GeoPackage-File exportieren

In [20]:
#Als GeoPackage-File exportieren
gdf.to_file('Daten/Gemeindeliste_1-3.gpkg', driver='GPKG', limit=0)
